### Stock Analysis

In this part we will make analysis using RSI, volatility and other basic financial tools.

In [22]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Creating a function to retrieve financial data from yfinance.

In [27]:
def get_stock_data(ticker, period='1y'):
    df = yf.download(ticker, period=period, auto_adjust=True)
    
    # Sütun isimlerindeki karmaşıklığı (MultiIndex) temizle
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.droplevel(1)
    
    # Getirileri Hesapla
    # Log Getiri (Analiz ve Volatilite için daha doğru)
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    return df

Creating a function that generates technical indicators.

In [28]:
def add_indicators(df):
    # Simple Moving Average (SMA 20 and 50)
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # RSI (Relative Strength Index)
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df

Creating a function to generate plot-candlestick

In [29]:
def plot_dashboard(df, ticker):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=(f'{ticker} Price Analysis', 'RSI Momentum'),
                        row_width=[0.2, 0.7])

    # Candlestick Graph
    fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close'], name='Fiyat'), row=1, col=1)

    # SMA Lines
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], line=dict(color='orange', width=1), name='SMA 20'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], line=dict(color='blue', width=1), name='SMA 50'), row=1, col=1)

    # 2nd Row: RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], line=dict(color='purple', width=2), name='RSI'), row=2, col=1)
    
    # Reference lines for RSI at 70 and 30
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # Layout adjustments
    fig.update_layout(title=f'{ticker} Technical Analysis Dashboard', yaxis_title='Price', xaxis_rangeslider_visible=False, height=800)
    fig.show()

## Using the functions:

In [34]:
symbol = "GARAN.IS" # Or "AAPL", "NVDA"

# 1. Get Data
data = get_stock_data(symbol)
print(data.head())

# 2. Add Indicators
data = add_indicators(data)

# 3. Plot Dashboard
plot_dashboard(data, symbol)

# 4. Show Last 5 Days
print(data[['Close', 'Log_Return', 'RSI']].tail())

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume  \
Date                                                                   
2024-12-10  124.303902  127.871985  123.628864  127.293373  15079399   
2024-12-11  121.507309  124.593206  118.228539  124.400340  25740589   
2024-12-12  121.314445  123.243133  120.350102  122.375222  18260905   
2024-12-13  123.243134  123.243134  119.192885  120.060796  17103441   
2024-12-16  123.243134  125.075381  122.085917  123.243134  21133440   

Price       Log_Return  
Date                    
2024-12-10         NaN  
2024-12-11   -0.022755  
2024-12-12   -0.001589  
2024-12-13    0.015773  
2024-12-16    0.000000  


Price            Close  Log_Return        RSI
Date                                         
2025-12-03  141.800003   -0.001409  78.082187
2025-12-04  137.399994   -0.031521  65.769216
2025-12-05  141.500000    0.029403  73.356382
2025-12-08  141.199997   -0.002122  71.428548
2025-12-09  140.600006   -0.004258  65.873008
